In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import ImageNormalize, ZScaleInterval, LinearStretch
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.nddata import Cutout2D
from reproject import reproject_interp

In [2]:
f160w = fits.open("id1j03020_drz.fits")
f555w = fits.open("id1j03010_drc_sci.ali.fits")
fors2 = fits.open("fors2.cutout.fits")

In [3]:
#Read the fors2 cutout WCS
fors2_wcs = WCS(fors2[0].header)

In [4]:
#We'll take as a reference point the central pixel of the FORS2 cutout and its coordinates. 
ref_px = int((fors2[0].header['NAXIS1']-1)/2)
ref_py = int((fors2[0].header['NAXIS2']-1)/2)
ref_coords = fors2_wcs.pixel_to_world(ref_px, ref_py)

In [5]:
for band in ["F160W", "F555W"]:
    
    if band == "F160W":
        im = f160w['SCI']
    else:
        im = f555w[0]

    #Read the WCS
    band_wcs = WCS(im.header)

    #Get the closest pixel to the reference position from the FORS2 cutout, and get its coordinates. These are the reference points we will use for the new WCS. 
    px, py = band_wcs.world_to_pixel(ref_coords)
    px = int(np.round(px,0))
    py = int(np.round(py,0))
    cen_coords = band_wcs.pixel_to_world(px,py)    

    #Set up the new header WCS with the updated reference points and rotated so that North is up. 
    test_header = band_wcs.to_header()
    test_header['PC1_1'] = -band_wcs.proj_plane_pixel_scales()[0].value
    test_header['PC1_2'] = 0.
    test_header['PC2_1'] = 0.
    test_header['PC2_2'] = band_wcs.proj_plane_pixel_scales()[0].value
    test_header['CRVAL1'] = cen_coords.ra.deg
    test_header['CRVAL2'] = cen_coords.dec.deg
    test_header['CRPIX1'] = px+1
    test_header['CRPIX2'] = py+1
    test_wcs = WCS(test_header)

    #Reproject the image onto the new WCS.
    im_rep, footprint = reproject_interp(im, test_wcs, shape_out=im.data.shape)
    fits.writeto("{}.reproj.fits".format(band), im_rep, header=test_header, overwrite=True)

    #Finally, make the cutout matched in angular size and center to the FORS2 one. 
    dy, dx = fors2[0].data.shape
    dx_band_cutout = dx * fors2_wcs.proj_plane_pixel_scales()[0]/test_wcs.proj_plane_pixel_scales()[0]
    dy_band_cutout = dy * fors2_wcs.proj_plane_pixel_scales()[1]/test_wcs.proj_plane_pixel_scales()[1]
    dx_band_cutout = int(np.round(dx_band_cutout,0))
    dy_band_cutout = int(np.round(dy_band_cutout,0))
    band_cutout = Cutout2D(im_rep, cen_coords, (dy_band_cutout, dx_band_cutout), wcs=test_wcs)
    fits.writeto("{}.cutout.fits".format(band),band_cutout.data, header=band_cutout.wcs.to_header() ,overwrite=True)
    

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
